In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

import pandas as pd
import gluonnlp as nlp
import numpy as np


from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel
from transformers import AdamW

In [2]:
from kobert_hf.kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

In [3]:
tokenizer.vocab_file

'C:\\Users\\cpprh/.cache\\huggingface\\transformers\\6920ce54223b52af14e36b32047ced34c47ec88ac51f45ce0141aaa1054e3263.7eed87d19282a93a2d45e130f20b4d8e831cbf8e957f1476628fd4ab99ae977f'

In [4]:
from kobert.pytorch_kobert import get_pytorch_kobert_model
bertmodel, vocab = get_pytorch_kobert_model('C:\\Users\\cpprh/.cache\\huggingface\\transformers', '.cache') 

using cached model. C:\Users\cpprh\Documents\GitHub\EatShare-AI\.cache\kobert_v1.zip
using cached model. C:\Users\cpprh\Documents\GitHub\EatShare-AI\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [5]:
bertmodel

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

In [10]:
data = pd.read_csv('./final.csv', encoding='utf8')

In [11]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [12]:
data

,DESC_KOR,label
0,굴국밥,0
1,물회,0
2,생선물회,0
3,닭칼국수,0
4,삼선자장면,0
...,...,...
78193,닭고기표고버섯렌틸콩진밥 밀키트,26
78194,설성목장 한우 사골육수 떡볶이,26
78195,우둔 스테이크 도시락,26
78196,"표고버섯, 배지재배, 갓, 말린것",26


In [13]:
data_list = []
for quest, label in zip(data['DESC_KOR'], data['label']):
    data=[]
    data.append(quest)
    data.append(str(label))
    
    data_list.append(data)

In [14]:
data_list, len(data_list)

([['굴국밥', '0'],
  ['물회', '0'],
  ['생선물회', '0'],
  ['닭칼국수', '0'],
  ['삼선자장면', '0'],
  ['우동(중식)', '0'],
  ['해물크림소스스파게티', '0'],
  ['돼지국밥', '0'],
  ['잡탕밥', '0'],
  ['장어덮밥', '0'],
  ['참치덮밥', '0'],
  ['해물덮밥', '0'],
  ['회덮밥', '0'],
  ['치즈피자', '0'],
  ['페퍼로니피자', '0'],
  ['돼지등갈비찜', '0'],
  ['붕어찜', '0'],
  ['안동찜닭', '0'],
  ['치킨데리야끼', '0'],
  ['미트볼 토마토 스파게티', '0'],
  ['쟁반국수', '0'],
  ['콩국수', '0'],
  ['회냉면', '0'],
  ['돼지머리국밥', '0'],
  ['모듬회덮밥', '0'],
  ['비빔밥', '0'],
  ['제육덮밥', '0'],
  ['짬뽕밥', '0'],
  ['오리고기죽', '0'],
  ['부대찌개', '0'],
  ['클래식치즈피자', '0'],
  ['햄앤체다피자', '0'],
  ['고르곤졸라피자', '0'],
  ['페파로니', '0'],
  ['페파로니매니아피자', '0'],
  ['청양페파로니피자', '0'],
  ['하와이안피자', '0'],
  ['베이컨포테이토피자', '0'],
  ['불고기피자', '0'],
  ['부라타치즈피자', '0'],
  ['나폴리슈림프피자', '0'],
  ['디아볼라피자', '0'],
  ['볼로네즈', '0'],
  ['고향 만두', '0'],
  ['신 비비고 한섬 만두', '0'],
  ['서원 교자만두', '0'],
  ['밴쯔 덤플링', '0'],
  ['매생이 삼계탕', '0'],
  ['더욱 맛있어진 뉴 백설군만두', '0'],
  ['청고추만두', '0'],
  ['알찬소시지', '0'],
  ['전통 우리만두', '0'],
  ['맘마밀1', '0'],
  ['맘마밀2', '0'],

In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [16]:
max_len = 64
batch_size = 4
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

device = torch.device("cuda:0")

In [17]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=10)

In [18]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=27,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch: {}, batch id: {}, loss: {}, train acc: {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch: {}, train acc: {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch: {}, test acc: {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|                                                                              | 3/15640 [00:00<1:58:17,  2.20it/s]

epoch: 1, batch id: 1, loss: 3.1744022369384766, train acc: 0.0


  1%|█                                                                             | 203/15640 [00:15<21:03, 12.21it/s]

epoch: 1, batch id: 201, loss: 3.235738515853882, train acc: 0.1318407960199005


  3%|█▉                                                                            | 401/15640 [00:32<22:13, 11.43it/s]

...

 87%|██████████████████████████████████████████████████████████████████          | 13602/15640 [18:46<02:57, 11.48it/s]

epoch: 2, batch id: 13601, loss: 1.389037013053894, train acc: 0.4377251672671127


 88%|███████████████████████████████████████████████████████████████████         | 13801/15640 [19:02<02:47, 11.00it/s]

epoch: 2, batch id: 13801, loss: 1.5948200225830078, train acc: 0.4386638649373234


 89%|████████████████████████████████████████████████████████████████████        | 13996/15640 [19:19<02:02, 13.43it/s]

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("탄2, 단2, 지2")
            elif np.argmax(logits) == 1:
                test_eval.append("탄1, 단1, 지1")
            elif np.argmax(logits) == 2:
                test_eval.append("탄0, 단0, 지0")
            else:
                test_eval.append("응애")
        print(">> " + test_eval[0])